In [1]:
import pandas as pd
import openml
from openml.datasets import edit_dataset, fork_dataset, get_dataset

data_name = 'abalone'

data = openml.datasets.get_dataset(720)

train_X, labels, categorical_indicator, attribute_names = data.get_data(
    target=data.default_target_attribute, dataset_format="dataframe"
)

In [2]:
import shapreg  # https://github.com/iancovert/shapley-regression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
nominal = [b for a, b in zip(categorical_indicator, attribute_names) if a]
dummied_data = pd.get_dummies(train_X, columns=nominal)

dummied_data.fillna(0, inplace=True)
dummied_data

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Sex_M,Sex_F,Sex_I
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,1,0,0
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,1,0,0
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,0,1,0
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,1,0,0
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,0,0,1
...,...,...,...,...,...,...,...,...,...,...
4172,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,0,1,0
4173,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,1,0,0
4174,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,1,0,0
4175,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,0,1,0


In [4]:
X_train, X_test, Y_train_, Y_test_ = train_test_split(dummied_data, labels, test_size=0.2, random_state=42)

X_train, X_val, Y_train_, Y_val_ = train_test_split(
    X_train, Y_train_, test_size=0.2, random_state=0)

# Data scaling
num_features = X_train.shape[1]
feature_names = X_train.columns.tolist()
ss = StandardScaler()
ss.fit(X_train)
X_train_norm = ss.transform(X_train)
X_val_norm = ss.transform(X_val)
X_test_norm = ss.transform(X_test)

In [5]:
print(X_train_norm.shape, X_val_norm.shape, X_test_norm.shape)

(2672, 10) (669, 10) (836, 10)


In [6]:
Y_train = pd.get_dummies(Y_train_)
Y_test = pd.get_dummies(Y_test_)
Y_val = pd.get_dummies(Y_val_)

In [7]:
Y_train.shape

(2672, 2)

In [8]:
Y_train_

2168    P
536     N
1180    N
1244    P
3363    N
       ..
542     N
2695    P
3612    N
4061    N
3966    P
Name: binaryClass, Length: 2672, dtype: category
Categories (2, object): ['P' < 'N']

In [9]:
mapping = {'P': 1, 'N': 0}


Y_train_ = Y_train_.replace(mapping)
Y_test_ = Y_test_.replace(mapping)
Y_val_ = Y_val_.replace(mapping)

In [10]:
Y_train_

2168    1
536     0
1180    0
1244    1
3363    0
       ..
542     0
2695    1
3612    0
4061    0
3966    1
Name: binaryClass, Length: 2672, dtype: int64

In [11]:
import xgboost as xgb
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score

xg_boost = xgb.XGBClassifier(objective='binary:logistic', use_label_encoder=False, 
                             learning_rate=0.01, n_estimators= 100, reg_lambda=0.001)

xgb_clf = xg_boost.fit(X_train_norm, Y_train_)
val_acc = xgb_clf.score(X_val_norm, Y_val_)
val_pred = xgb_clf.predict(X_val_norm)
f_score = f1_score(Y_val_, val_pred, average='macro')
prec = precision_score(Y_val_, val_pred, average='macro')
recall = recall_score(Y_val_, val_pred, average='macro')

print(f'XGB test_acc: {val_acc}\nrecall: {recall}\nprecision: {prec}\nf_score: {f_score}\n')

[10:09:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGB test_acc: 0.7757847533632287
recall: 0.774387515882532
precision: 0.7768398815542394
f_score: 0.7747656755009696



In [13]:
import torch
import torch.nn as nn
from fastshap.utils import MaskLayer1d
from fastshap import Surrogate, KLDivLoss
from ff_shap_training import BlackBoxWrapper, FF_SHAP_Training

In [14]:
bb = BlackBoxWrapper(xgb_clf, ss, num_features)

def imputer(x, S):
    x = torch.tensor(x, dtype=torch.float32, device=device)
    S = torch.tensor(S, dtype=torch.float32, device=device)
    pred = bb(x, S)
    return pred.cpu().data.numpy()

In [15]:
from tqdm import tqdm
import pickle
import os.path

#Generate Explanations for test data

if os.path.isfile(f'{data_name}/{data_name}_unbiased_exp.pickle'):
    print('Loading saved explanations')
    with open(f'{data_name}/{data_name}_unbiased_exp.pickle', 'rb') as file:
        unbiased_exp = pickle.load(file)
    with open(f'{data_name}/{data_name}_kernel_exp.pickle', 'rb') as file:
        kernel_exp = pickle.load(file)
        
else:
    unbiased_exp = []
    kernel_exp = []
    for i in tqdm(range(len(X_test_norm))):
        game = shapreg.games.PredictionGame(imputer, X_test_norm[i])
        tshap_values, all_results = shapreg.shapley.ShapleyRegression(
            game, batch_size=64, paired_sampling=True, detect_convergence=True,
            bar=True, return_all=True)
        unbiased_exp.append(tshap_values)
        kernel_exp.append(all_results)

    with open(f'{data_name}/{data_name}_unbiased_exp.pickle', 'wb') as file:
        pickle.dump(unbiased_exp, file)
    
    with open(f'{data_name}/{data_name}_kernel_exp.pickle', 'wb') as file:
        pickle.dump(kernel_exp, file)

Loading saved explanations


# FastSHAP

In [16]:
from fastshap import FastSHAP

device = torch.device('cuda')

# Check for model
if os.path.isfile(f'{data_name}/{data_name}_explainer_fastSHAP_bb.pt'):
    print('Loading saved explainer model')
    explainer = torch.load(f'{data_name}/{data_name}_explainer_fastSHAP_bb.pt').to(device)
    fastshap_bb = FastSHAP(explainer, bb, normalization='additive', link=None)

else:
    # Create explainer model
    explainer = nn.Sequential(
        nn.Linear(num_features, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, Y_train.shape[1] * num_features)).to(device)

    # Set up FastSHAP object
    fastshap_bb = FastSHAP(explainer, bb, normalization='additive', link=None)

    # Train
    fastshap_bb.train(
        X_train_norm,
        X_val_norm[:100],
        batch_size=64,
        num_samples=64,
        max_epochs=400,
        validation_samples=128,
        verbose=True,
        lookback=5)
    
    # Save explainer
    explainer.cpu()
    torch.save(explainer, f'{data_name}/{data_name}_explainer_fastSHAP_bb.pt')
    explainer.to(device)

Loading saved explainer model


In [17]:
from numpy.linalg import norm
from scipy.stats import spearmanr

test_pred = xgb_clf.predict(X_test_norm)
fastshap_values = fastshap_bb.shap_values(X_test_norm)

pred_fastshap_values = np.array([fastshap_values[i] for i, l in enumerate(test_pred)])
unbiased_values = np.array([unbiased_exp[i].values for i, l in enumerate(test_pred)])

l2_dis = []
cos_sim = []
spearman_c = []

for i in range(len(test_pred)):
    fastshap_vs = pred_fastshap_values[i].reshape(-1)
    unbiased_vs = unbiased_values[i].reshape(-1)

    l2_dis.append(norm(unbiased_vs - fastshap_vs))
    cos_sim.append(np.dot(unbiased_vs, fastshap_vs)/(norm(unbiased_vs)*norm(fastshap_vs)))
    coef, p = spearmanr(unbiased_vs, fastshap_vs)
    spearman_c.append(coef)
    
mean_l2_dis = np.mean(l2_dis, axis=0)
mean_cos_sim = np.mean(cos_sim, axis=0)
mean_spearman_c = np.mean(spearman_c, axis=0)

print(f'l2 distance: {mean_l2_dis}\ncosine sim: {mean_cos_sim}\nspearman corr: {mean_spearman_c}')

l2 distance: 0.08552393905503626
cosine sim: 0.9480855880219956
spearman corr: 0.8101521746951109


# FF-SHAP

In [18]:
#Generate Explanations for training data

if os.path.isfile(f'{data_name}/{data_name}_training_exp.pickle'):
    print('Loading saved explanations')
    with open(f'{data_name}/{data_name}_training_exp.pickle', 'rb') as file:
        training_exp = pickle.load(file)
        
else:
    training_exp = []
    for i in tqdm(range(len(X_train_norm))):
        game = shapreg.games.PredictionGame(imputer, X_train_norm[i])
        tshap_values, all_results = shapreg.shapley.ShapleyRegression(
            game, batch_size=256, paired_sampling=True, detect_convergence=True,
            bar=False, return_all=True)
        training_exp.append(tshap_values)
        
    with open(f'{data_name}/{data_name}_training_exp.pickle', 'wb') as file:
        pickle.dump(training_exp, file)


Loading saved explanations


In [19]:
#Generate Explanations for development data

if os.path.isfile(f'{data_name}/{data_name}_val_exp.pickle'):
    print('Loading saved explanations')
    with open(f'{data_name}/{data_name}_val_exp.pickle', 'rb') as file:
        val_exp = pickle.load(file)

else:
    val_exp = []
    for i in tqdm(range(len(X_val_norm))):
        game = shapreg.games.PredictionGame(imputer, X_val_norm[i])
        tshap_values, all_results = shapreg.shapley.ShapleyRegression(
            game, batch_size=256, paired_sampling=True, detect_convergence=True,
            bar=False, return_all=True)
        val_exp.append(tshap_values)
        
    with open(f'{data_name}/{data_name}_val_exp.pickle', 'wb') as file:
        pickle.dump(val_exp, file)

Loading saved explanations


In [20]:
train_targets = np.array([training_exp[i].values.reshape(-1) for i, l in enumerate(X_train_norm)])
val_targets = np.array([val_exp[i].values.reshape(-1) for i, l in enumerate(X_val_norm)])

# Training on full data

In [22]:
from ff_shap_training import BlackBoxWrapper, FF_SHAP_Training

device = torch.device('cuda')

# Check for model
if os.path.isfile(f'{data_name}/{data_name}_sampling_fastSHAP.pt'):
    print('Loading saved explainer model')
    explainer = torch.load(f'{data_name}/{data_name}_sampling_fastSHAP.pt').to(device)
    ex_fastshap = FF_SHAP_Training(explainer, bb, num_features, Y_train.shape[1])

else:
    # Create explainer model
    explainer = nn.Sequential(
        nn.Linear(num_features+Y_train.shape[1], 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, Y_train.shape[1] * num_features)).to(device)

    # Set up FastSHAP object
    ex_fastshap = FF_SHAP_Training(explainer, bb, num_features, Y_train.shape[1])

    # Train
    ex_fastshap.train(
        X_train_norm,
        train_targets,
        X_val_norm[:512],
        val_targets[:512],
        batch_size=64,
        num_samples=64,
        max_epochs=400,
        lr=2e-3,
        verbose=True,
        sampling=False,
        lookback=5)
    
    # Save explainer
    explainer.cpu()
    torch.save(explainer, f'{data_name}/{data_name}_sampling_fastSHAP.pt')
    explainer.to(device)

Loading saved explainer model


In [23]:
from numpy.linalg import norm
from scipy.stats import spearmanr

test_pred = xgb_clf.predict(X_test_norm)
fastshap_values = ex_fastshap.predict(X_test_norm).cpu().data.numpy()

pred_fastshap_values = np.array([fastshap_values[i] for i, l in enumerate(test_pred)])
unbiased_values = np.array([unbiased_exp[i].values for i, l in enumerate(test_pred)])

l2_dis = []
cos_sim = []
spearman_c = []

for i in range(len(test_pred)):
    fastshap_vs = pred_fastshap_values[i].reshape(-1)
    unbiased_vs = unbiased_values[i].reshape(-1)

    l2_dis.append(norm(unbiased_vs - fastshap_vs))
    cos_sim.append(np.dot(unbiased_vs, fastshap_vs)/(norm(unbiased_vs)*norm(fastshap_vs)))
    coef, p = spearmanr(unbiased_vs, fastshap_vs)
    spearman_c.append(coef)
    
mean_l2_dis = np.mean(l2_dis, axis=0)
mean_cos_sim = np.mean(cos_sim, axis=0)
mean_spearman_c = np.mean(spearman_c, axis=0)

print(f'l2 distance: {mean_l2_dis}\ncosine sim: {mean_cos_sim}\nspearman corr: {mean_spearman_c}')

l2 distance: 0.08359368275257573
cosine sim: 0.9782462787304702
spearman corr: 0.8609885958916429


# Training using 60% of the data

In [22]:
ind = int(np.ceil(len(X_train_norm)*0.6))

1604

In [24]:
from ff_shap_training import BlackBoxWrapper, FF_SHAP_Training

device = torch.device('cuda')

# Check for model
if os.path.isfile(f'{data_name}/{data_name}_60_ffastSHAP.pt'):
    print('Loading saved explainer model')
    explainer = torch.load(f'{data_name}/{data_name}_60_ffastSHAP.pt').to(device)
    ex_fastshap = FF_SHAP_Training(explainer, bb, num_features, Y_train.shape[1])

else:
    # Create explainer model
    explainer = nn.Sequential(
        nn.Linear(num_features+Y_train.shape[1], 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, Y_train.shape[1] * num_features)).to(device)

    # Set up FastSHAP object
    ex_fastshap = FF_SHAP_Training(explainer, bb, num_features, Y_train.shape[1])

    # Train
    ex_fastshap.train(
        X_train_norm[:ind],
        train_targets[:ind],
        X_val_norm[:512],
        val_targets[:512],
        batch_size=64,
        num_samples=64,
        max_epochs=400,
        lr=2e-3,
        verbose=True,
        sampling=False,
        lookback=5)
    
    # Save explainer
    explainer.cpu()
    torch.save(explainer, f'{data_name}/{data_name}_60_ffastSHAP.pt')
    explainer.to(device)

Loading saved explainer model


In [25]:
from numpy.linalg import norm
from scipy.stats import spearmanr

test_pred = xgb_clf.predict(X_test_norm)
fastshap_values = ex_fastshap.predict(X_test_norm).cpu().data.numpy()

pred_fastshap_values = np.array([fastshap_values[i] for i, l in enumerate(test_pred)])
unbiased_values = np.array([unbiased_exp[i].values for i, l in enumerate(test_pred)])

l2_dis = []
cos_sim = []
spearman_c = []

for i in range(len(test_pred)):
    fastshap_vs = pred_fastshap_values[i].reshape(-1)
    unbiased_vs = unbiased_values[i].reshape(-1)

    l2_dis.append(norm(unbiased_vs - fastshap_vs))
    cos_sim.append(np.dot(unbiased_vs, fastshap_vs)/(norm(unbiased_vs)*norm(fastshap_vs)))
    coef, p = spearmanr(unbiased_vs, fastshap_vs)
    spearman_c.append(coef)
    
mean_l2_dis = np.mean(l2_dis, axis=0)
mean_cos_sim = np.mean(cos_sim, axis=0)
mean_spearman_c = np.mean(spearman_c, axis=0)

print(f'l2 distance: {mean_l2_dis}\ncosine sim: {mean_cos_sim}\nspearman corr: {mean_spearman_c}')

l2 distance: 0.08870551835738033
cosine sim: 0.9726661084617213
spearman corr: 0.8509137676727704


# Training using 30% of the data

In [26]:
ind = int(np.ceil(len(X_train_norm)*0.3))

In [27]:
from ff_shap_training import BlackBoxWrapper, FF_SHAP_Training

device = torch.device('cuda')

# Check for model
if os.path.isfile(f'{data_name}/{data_name}_30_ffastSHAP.pt'):
    print('Loading saved explainer model')
    explainer = torch.load(f'{data_name}/{data_name}_30_ffastSHAP.pt').to(device)
    ex_fastshap = FF_SHAP_Training(explainer, bb, num_features, Y_train.shape[1])

else:
    # Create explainer model
    explainer = nn.Sequential(
        nn.Linear(num_features+Y_train.shape[1], 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, Y_train.shape[1] * num_features)).to(device)

    # Set up FastSHAP object
    ex_fastshap = FF_SHAP_Training(explainer, bb, num_features, Y_train.shape[1])

    # Train
    ex_fastshap.train(
        X_train_norm[:ind],
        train_targets[:ind],
        X_val_norm[:512],
        val_targets[:512],
        batch_size=64,
        num_samples=64,
        max_epochs=400,
        lr=2e-3,
        verbose=True,
        sampling=False,
        lookback=5)
    
    # Save explainer
    explainer.cpu()
    torch.save(explainer, f'{data_name}/{data_name}_30_ffastSHAP.pt')
    explainer.to(device)

Loading saved explainer model


In [28]:
from numpy.linalg import norm
from scipy.stats import spearmanr

test_pred = xgb_clf.predict(X_test_norm)
fastshap_values = ex_fastshap.predict(X_test_norm).cpu().data.numpy()

pred_fastshap_values = np.array([fastshap_values[i] for i, l in enumerate(test_pred)])
unbiased_values = np.array([unbiased_exp[i].values for i, l in enumerate(test_pred)])

l2_dis = []
cos_sim = []
spearman_c = []

for i in range(len(test_pred)):
    fastshap_vs = pred_fastshap_values[i].reshape(-1)
    unbiased_vs = unbiased_values[i].reshape(-1)

    l2_dis.append(norm(unbiased_vs - fastshap_vs))
    cos_sim.append(np.dot(unbiased_vs, fastshap_vs)/(norm(unbiased_vs)*norm(fastshap_vs)))
    coef, p = spearmanr(unbiased_vs, fastshap_vs)
    spearman_c.append(coef)
    
mean_l2_dis = np.mean(l2_dis, axis=0)
mean_cos_sim = np.mean(cos_sim, axis=0)
mean_spearman_c = np.mean(spearman_c, axis=0)

print(f'l2 distance: {mean_l2_dis}\ncosine sim: {mean_cos_sim}\nspearman corr: {mean_spearman_c}')

l2 distance: 0.10368585405717788
cosine sim: 0.9649732049229227
spearman corr: 0.827251142209591


# Training using 15% of the data

In [29]:
ind = int(np.ceil(len(X_train_norm)*0.15))

In [30]:
from ff_shap_training import BlackBoxWrapper, FF_SHAP_Training

device = torch.device('cuda')

# Check for model
if os.path.isfile(f'{data_name}/{data_name}_15_ffastSHAP.pt'):
    print('Loading saved explainer model')
    explainer = torch.load(f'{data_name}/{data_name}_15_ffastSHAP.pt').to(device)
    ex_fastshap = FF_SHAP_Training(explainer, bb, num_features, Y_train.shape[1])

else:
    # Create explainer model
    explainer = nn.Sequential(
        nn.Linear(num_features+Y_train.shape[1], 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, Y_train.shape[1] * num_features)).to(device)

    # Set up FastSHAP object
    ex_fastshap = FF_SHAP_Training(explainer, bb, num_features, Y_train.shape[1])

    # Train
    ex_fastshap.train(
        X_train_norm[:ind],
        train_targets[:ind],
        X_val_norm[:512],
        val_targets[:512],
        batch_size=64,
        num_samples=64,
        max_epochs=400,
        lr=2e-3,
        verbose=True,
        sampling=False,
        lookback=5)
    
    # Save explainer
    explainer.cpu()
    torch.save(explainer, f'{data_name}/{data_name}_15_ffastSHAP.pt')
    explainer.to(device)

Loading saved explainer model


In [31]:
from numpy.linalg import norm
from scipy.stats import spearmanr

test_pred = xgb_clf.predict(X_test_norm)
fastshap_values = ex_fastshap.predict(X_test_norm).cpu().data.numpy()

pred_fastshap_values = np.array([fastshap_values[i] for i, l in enumerate(test_pred)])
unbiased_values = np.array([unbiased_exp[i].values for i, l in enumerate(test_pred)])

l2_dis = []
cos_sim = []
spearman_c = []

for i in range(len(test_pred)):
    fastshap_vs = pred_fastshap_values[i].reshape(-1)
    unbiased_vs = unbiased_values[i].reshape(-1)

    l2_dis.append(norm(unbiased_vs - fastshap_vs))
    cos_sim.append(np.dot(unbiased_vs, fastshap_vs)/(norm(unbiased_vs)*norm(fastshap_vs)))
    coef, p = spearmanr(unbiased_vs, fastshap_vs)
    spearman_c.append(coef)
    
mean_l2_dis = np.mean(l2_dis, axis=0)
mean_cos_sim = np.mean(cos_sim, axis=0)
mean_spearman_c = np.mean(spearman_c, axis=0)

print(f'l2 distance: {mean_l2_dis}\ncosine sim: {mean_cos_sim}\nspearman corr: {mean_spearman_c}')

l2 distance: 0.14637177174409305
cosine sim: 0.9529542708494543
spearman corr: 0.8027754793682771


# Training with MSE loss

In [32]:
from ff_shap_training import BlackBoxWrapper, FF_SHAP_Training

device = torch.device('cuda')

# Check for model
if os.path.isfile(f'{data_name}/{data_name}_MSE_fastSHAP.pt'):
    print('Loading saved explainer model')
    explainer = torch.load(f'{data_name}/{data_name}_MSE_fastSHAP.pt').to(device)
    ex_fastshap = FF_SHAP_Training(explainer, bb, num_features, Y_train.shape[1])

else:
    # Create explainer model
    explainer = nn.Sequential(
        nn.Linear(num_features+Y_train.shape[1], 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, Y_train.shape[1] * num_features)).to(device)

    # Set up FastSHAP object
    ex_fastshap = FF_SHAP_Training(explainer, bb, num_features, Y_train.shape[1])

    # Train
    ex_fastshap.train(
        X_train_norm,
        train_targets,
        X_val_norm[:512],
        val_targets[:512],
        batch_size=64,
        num_samples=64,
        max_epochs=400,
        loss_func='MSE',
        lr=2e-3,
        verbose=True,
        sampling=False,
        lookback=5)
    
    # Save explainer
    explainer.cpu()
    torch.save(explainer, f'{data_name}/{data_name}_MSE_fastSHAP.pt')
    explainer.to(device)

Loading saved explainer model


In [33]:
from numpy.linalg import norm
from scipy.stats import spearmanr

test_pred = xgb_clf.predict(X_test_norm)
fastshap_values = ex_fastshap.predict(X_test_norm).cpu().data.numpy()

pred_fastshap_values = np.array([fastshap_values[i] for i, l in enumerate(test_pred)])
unbiased_values = np.array([unbiased_exp[i].values for i, l in enumerate(test_pred)])

l2_dis = []
cos_sim = []
spearman_c = []

for i in range(len(test_pred)):
    fastshap_vs = pred_fastshap_values[i].reshape(-1)
    unbiased_vs = unbiased_values[i].reshape(-1)

    l2_dis.append(norm(unbiased_vs - fastshap_vs))
    cos_sim.append(np.dot(unbiased_vs, fastshap_vs)/(norm(unbiased_vs)*norm(fastshap_vs)))
    coef, p = spearmanr(unbiased_vs, fastshap_vs)
    spearman_c.append(coef)
    
mean_l2_dis = np.mean(l2_dis, axis=0)
mean_cos_sim = np.mean(cos_sim, axis=0)
mean_spearman_c = np.mean(spearman_c, axis=0)

print(f'l2 distance: {mean_l2_dis}\ncosine sim: {mean_cos_sim}\nspearman corr: {mean_spearman_c}')

l2 distance: 0.05556413335936369
cosine sim: 0.9719006268131446
spearman corr: 0.8565600604381767


# Training without augmentation

In [34]:
from ff_shap_training import BlackBoxWrapper, FF_SHAP_Training

device = torch.device('cuda')

# Check for model
if os.path.isfile(f'{data_name}/{data_name}_NAug_fastSHAP.pt'):
    print('Loading saved explainer model')
    explainer = torch.load(f'{data_name}/{data_name}_NAug_fastSHAP.pt').to(device)
    ex_fastshap = FF_SHAP_Training(explainer, bb, num_features, Y_train.shape[1], augmentation=False)

else:
    # Create explainer model
    explainer = nn.Sequential(
        nn.Linear(num_features, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, Y_train.shape[1] * num_features)).to(device)

    # Set up FastSHAP object
    ex_fastshap = FF_SHAP_Training(explainer, bb, num_features, Y_train.shape[1], False)

    # Train
    ex_fastshap.train(
        X_train_norm,
        train_targets,
        X_val_norm[:512],
        val_targets[:512],
        batch_size=64,
        num_samples=64,
        max_epochs=400,
        loss_func='cosine',
        lr=2e-3,
        verbose=True,
        sampling=False,
        lookback=5)
    
    # Save explainer
    explainer.cpu()
    torch.save(explainer, f'{data_name}/{data_name}_NAug_fastSHAP.pt')
    explainer.to(device)

Loading saved explainer model


In [35]:
from numpy.linalg import norm
from scipy.stats import spearmanr

test_pred = xgb_clf.predict(X_test_norm)
fastshap_values = ex_fastshap.predict(X_test_norm).cpu().data.numpy()

pred_fastshap_values = np.array([fastshap_values[i] for i, l in enumerate(test_pred)])
unbiased_values = np.array([unbiased_exp[i].values for i, l in enumerate(test_pred)])

l2_dis = []
cos_sim = []
spearman_c = []

for i in range(len(test_pred)):
    fastshap_vs = pred_fastshap_values[i].reshape(-1)
    unbiased_vs = unbiased_values[i].reshape(-1)

    l2_dis.append(norm(unbiased_vs - fastshap_vs))
    cos_sim.append(np.dot(unbiased_vs, fastshap_vs)/(norm(unbiased_vs)*norm(fastshap_vs)))
    coef, p = spearmanr(unbiased_vs, fastshap_vs)
    spearman_c.append(coef)
    
mean_l2_dis = np.mean(l2_dis, axis=0)
mean_cos_sim = np.mean(cos_sim, axis=0)
mean_spearman_c = np.mean(spearman_c, axis=0)

print(f'l2 distance: {mean_l2_dis}\ncosine sim: {mean_cos_sim}\nspearman corr: {mean_spearman_c}')

l2 distance: 0.11100849499733473
cosine sim: 0.9672739921852314
spearman corr: 0.8428013814440407
